```{note} Note
Simple moving Average (SMA) using Talib library

```

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import cycle

import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"

from pathlib import Path
from tqdm.auto import tqdm


from sklearn.metrics import mean_absolute_error
from IPython.display import display, HTML
# %load_ext autoreload
# %autoreload 2


np.random.seed()
tqdm.pandas()

In [2]:
from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
import yfinance as yf
from datetime import date, datetime, timedelta

from ta.utils import dropna
from ta.momentum import RSIIndicator

import talib as tb

In [5]:

def format_plot(fig, legends = None, xlabel="Time", ylabel="Value", figsize=(400,900), 
                font_size=15, title_font_size=20, text=None):
    if legends:
        names = cycle(legends)
        fig.for_each_trace(lambda t:  t.update(name = next(names)))
    fig.update_layout(
            autosize=False,
            width=figsize[1],
            height=figsize[0],
            title={
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            titlefont={
                "size": 20
            },
            legend_title = None,
            legend=dict(
                font=dict(size=font_size),
                orientation="h",
                yanchor="bottom",
                y=0.98,
                xanchor="right",
                x=1,
            ),
            yaxis=dict(
                title_text=ylabel,
                titlefont=dict(size=font_size),
                tickfont=dict(size=font_size),
            ),
            xaxis=dict(
                title_text=xlabel,
                titlefont=dict(size=font_size),
                tickfont=dict(size=font_size),
            ),
            annotations = [dict(
                x=-0.1,
                y=-0.3,
                xref='paper',
                yref='paper',
                text = text,
                showarrow = False,
                font= {"size":15, "family":"Arial Black"}

            )]
        )
    return fig



In [6]:
riskyAsset = "AMZN"
startdate = date.today() - timedelta(days=365)

df = yf.download(riskyAsset, start=startdate)

[*********************100%%**********************]  1 of 1 completed


In [7]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-10-11,112.709999,115.480003,110.389999,112.209999,112.209999,56432200
2022-10-12,112.489998,113.830002,111.400002,112.900002,112.900002,45728700
2022-10-13,107.879997,113.440002,105.349998,112.529999,112.529999,86868100
2022-10-14,114.099998,114.959999,106.599998,106.900002,106.900002,67737300
2022-10-17,110.110001,114.190002,110.089996,113.790001,113.790001,62782000
